In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os

In [3]:
userFeature_data = []
with open('data/userFeature.data', 'r') as f:
    cnt = 0
    for i, line in enumerate(f):
        line = line.strip().split('|')
        userFeature_dict = {}
        for each in line:
            each_list = each.split(' ')
            userFeature_dict[each_list[0]] = ' '.join(each_list[1:])
        userFeature_data.append(userFeature_dict)
        if i % 100000 == 0:
            print(i)
        if i % 1000000 == 0:
            user_feature = pd.DataFrame(userFeature_data)
            user_feature.to_csv('data/userFeature_' + str(cnt) + '.csv', index=False)
            cnt += 1
            del userFeature_data, user_feature
            userFeature_data = []
    user_feature = pd.DataFrame(userFeature_data)
    user_feature.to_csv('data/userFeature_' + str(cnt) + '.csv', index=False)
    del userFeature_data, user_feature
    user_feature = pd.concat(
        [pd.read_csv('data/userFeature_' + str(i) + '.csv') for i in range(cnt + 1)]).reset_index(drop=True)
    user_feature.to_csv('data/userFeature.csv', index=False)


0


100000


200000


300000


400000


...

11000000


11100000


11200000


11300000


11400000


In [2]:
train=pd.read_csv('data/train.csv')
predict=pd.read_csv('data/test.csv')
ad_feature = pd.read_csv('data/adFeature.csv')
user_feature = pd.read_csv('data/userFeature.csv')
print("read prepared!")
train.loc[train['label']==-1,'label']=0
predict['label']=-1
data=pd.concat([train,predict])
data=pd.merge(data,ad_feature,on='aid',how='left')
data=pd.merge(data,user_feature,on='uid',how='left')
data=data.fillna('-1')
one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

read prepared!


In [3]:
train = data[data.label != -1]
train_y = train.pop('label')

test = data[data.label == -1]
res = test[['aid', 'uid']]
test = test.drop('label', axis=1)

In [4]:
print(train.shape)
print(train_y.shape)
print(test.shape)
print(res.shape)

(8798814, 32)
(8798814,)
(2265989, 32)
(2265989, 2)


In [10]:
train.to_csv('data_0505/train_labeled.csv', index=False)
train_y.to_csv('data_0505/train_y.csv', index=False)
res.to_csv("data_0505/res.csv", index=False)
test.to_csv("data_0505/test_labeled.csv", index=False)

In [13]:
sparse.hstack((None, None))

<0x0 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in COOrdinate format>

In [2]:
train = pd.read_csv('data_0505/train_labeled.csv')
test = pd.read_csv('data_0505/test_labeled.csv')
one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']

In [4]:
enc = OneHotEncoder()
train_x = None
test_x = None
print("one hot..")
for feature in one_hot_feature:
    print(feature)
    enc.fit(train[feature].values.reshape(-1, 1))
    train_a = enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x = sparse.hstack((train_x, train_a), format='coo')
    test_x = sparse.hstack((test_x, test_a), format='coo')
print('one-hot prepared !')
sparse.save_npz("data_0505/train_x_one_hot.npz", train_x)
sparse.save_npz("data_0505/test_x_one_hot.npz", test_x)

one hot..
creativeSize


LBS


age


carrier


consumptionAbility


...

creativeId


adCategoryId


productId


productType


one-hot prepared !


In [3]:
train = pd.read_csv('data_0505/train_labeled.csv')
test = pd.read_csv('data_0505/test_labeled.csv')
train_x = sparse.load_npz('data_0505/train_x_one_hot.npz')
test_x = sparse.load_npz('data_0505/test_x_one_hot.npz')
one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature1=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5']
vector_feature2=['kw1','kw2','kw3','topic1','topic2','topic3']

In [7]:
train[vector_feature1].to_csv('data_0505/vector1.csv', index=False)
train[vector_feature2].to_csv('data_0505/vector2.csv', index=False)

In [9]:
test[vector_feature1].to_csv('data_0505/t_vector1.csv', index=False)
test[vector_feature2].to_csv('data_0505/t_vector2.csv', index=False)

In [2]:
train_x = sparse.load_npz('data_0505/train_x_one_hot.npz')
test_x = sparse.load_npz('data_0505/test_x_one_hot.npz')
tr_vec1 = pd.read_csv('data_0505/vector1.csv')
te_vec1 = pd.read_csv('data_0505/t_vector1.csv')
one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature1=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5']
vector_feature2=['kw1','kw2','kw3','topic1','topic2','topic3']

In [3]:
print(train_x.shape)
print(test_x.shape)
print(tr_vec1.shape)
print(te_vec1.shape)

(8798814, 1444)
(2265989, 1444)
(8798814, 7)
(2265989, 7)


In [10]:
cv = CountVectorizer(min_df=500)
print("vector..")
for feature in vector_feature1:
    print(feature)
    cv.fit(tr_vec1[feature])
    train_a = cv.transform(tr_vec1[feature])
    test_a = cv.transform(te_vec1[feature])
    train_x = sparse.hstack((train_x, train_a), format='coo')
    test_x = sparse.hstack((test_x, test_a), format='coo')
print('cv prepared !')
sparse.save_npz("data_0505/train_x_vec1.npz", train_x)
sparse.save_npz("data_0505/test_x_vec1.npz", test_x)

vector..
appIdAction


appIdInstall


interest1


interest2


interest3


interest4


interest5


cv prepared !


In [11]:
print(train_x.shape)
print(test_x.shape)
print(tr_vec1.shape)
print(te_vec1.shape)

(8798814, 6105)
(2265989, 6105)
(8798814, 7)
(2265989, 7)


In [2]:
train_x = sparse.load_npz('data_0505/train_x_vec1.npz')
test_x = sparse.load_npz('data_0505/test_x_vec1.npz')
tr_vec2 = pd.read_csv('data_0505/vector2.csv')
te_vec2 = pd.read_csv('data_0505/t_vector2.csv')
one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature1=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5']
vector_feature2=['kw1','kw2','kw3','topic1','topic2','topic3']

In [3]:
print(train_x.shape)
print(test_x.shape)
print(tr_vec2.shape)
print(te_vec2.shape)

(8798814, 6105)
(2265989, 6105)
(8798814, 6)
(2265989, 6)


In [5]:
cv = CountVectorizer(min_df=500)
print("vector..")
for feature in vector_feature2:
    print(feature)
    cv.fit(tr_vec2[feature])
    train_a = cv.transform(tr_vec2[feature])
    test_a = cv.transform(te_vec2[feature])
    train_x = sparse.hstack((train_x, train_a), format='coo')
    test_x = sparse.hstack((test_x, test_a), format='coo')
print('cv prepared !')
sparse.save_npz("data_0505/train_x.npz", train_x)
sparse.save_npz("data_0505/test_x.npz", test_x)

vector..
kw1


KeyboardInterrupt: 

In [ ]:
print(train_x.shape)
print(test_x.shape)
print(tr_vec2.shape)
print(te_vec2.shape)

In [2]:
train_x = sparse.load_npz('data_0505/train_x.npz')
test_x = sparse.load_npz('data_0505/test_x.npz')
train_ori = sparse.load_npz('data_0505/train_ori.npz')
test_ori = sparse.load_npz('data_0505/test_ori.npz')

FileNotFoundError: [Errno 2] No such file or directory: 'data_0505/train_ori.npz'

In [3]:
print(train_x.shape)
print(test_x.shape)
# print(train_ori.shape)
# print(test_ori.shape)

(8798814, 33872)
(2265989, 33872)


In [2]:
train_x = sparse.load_npz('data_0505/train_x.npz')
train_y = pd.read_csv('data_0505/train_y.csv')
train_y.shape

(8798814, 1)

In [3]:
tr_x, val_x, tr_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1111)
print(tr_x.shape)
print(val_x.shape)
print(tr_y.shape)
print(val_y.shape)

(7039051, 33872)
(1759763, 33872)
(7039051, 1)
(1759763, 1)


In [12]:
sparse.save_npz("data_0505/tr_x.npz", tr_x.tocoo())
sparse.save_npz("data_0505/val_x.npz", val_x.tocoo())
tr_y.to_csv("data_0505/tr_y.csv", index=False)
val_y.to_csv("data_0505/val_y.csv", index=False)

In [2]:
tr_x = sparse.load_npz('data_0505/tr_x.npz')
val_x = sparse.load_npz('data_0505/val_x.npz')
tr_y = pd.read_csv('data_0505/tr_y.csv')
val_y = pd.read_csv('data_0505/val_y.csv')

In [9]:
print(tr_x.shape)
print(val_x.shape)
print(tr_y.shape)
print(val_y.shape)

(7039051, 33872)
(1759763, 33872)
(7039051, 1)
(1759763, 1)


In [3]:
sparse.save_npz("data_l500/train_x.npz", train_x)
sparse.save_npz("data_l500/test_x.npz", test_x)

In [16]:
train=data[data.label!=-1]
train_y=train.pop('label')
train, evals, train_y, evals_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)

In [17]:
# test=data[data.label==-1]
# res=test[['aid','uid']]
# test=test.drop('label',axis=1)
enc = OneHotEncoder()
train_x=train[['creativeSize']]
# test_x=test[['creativeSize']]
evals_x=evals[['creativeSize']]
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
#     test_a = enc.transform(test[feature].values.reshape(-1, 1))
    evals_a=enc.transform(evals[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a))
    evals_x = sparse.hstack((evals_x, evals_a))
print('one-hot prepared !')

cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
#     test_a = cv.transform(test[feature])
    evals_a=cv.transform(evals[feature])
    train_x = sparse.hstack((train_x, train_a))
#     test_x = sparse.hstack((test_x, test_a))
    evals_x=sparse.hstack((evals_x,evals_a))
print('cv prepared !')

one-hot prepared !


AttributeError: module 'scipy.sparse' has no attribute 'hstach'

In [24]:
for feature in vector_feature[1:]:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
#     test_a = cv.transform(test[feature])
    evals_a=cv.transform(evals[feature])
    train_x = sparse.hstack((train_x, train_a))
#     test_x = sparse.hstack((test_x, test_a))
    evals_x=sparse.hstack((evals_x,evals_a))
print('cv prepared !')

cv prepared !


In [25]:
evals_x

<1759763x419652 sparse matrix of type '<class 'numpy.float64'>'
	with 117348565 stored elements in COOrdinate format>

In [26]:
train_x

<7039051x419652 sparse matrix of type '<class 'numpy.float64'>'
	with 469401979 stored elements in COOrdinate format>

In [27]:
sparse.save_npz("evals_x.npz",evals_x)

In [ ]:
print("LGB test")
clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
    max_depth=-1, n_estimators=10000, objective='binary',
    subsample=0.65, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=-1
)
clf.fit(train_x, train_y, eval_set=[(evals_x, evals_y)], eval_metric='auc',early_stopping_rounds=100)
res['score'] = clf.predict_proba(test_x)[:,1]
res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
res.to_csv('./submission2.csv', index=False)
os.system('zip ./baseline2.zip ./submission2.csv')


In [28]:
res=test[['aid','uid']]


In [30]:
res.to_csv("res.csv",index=False)

In [33]:
train_y.to_csv('train_y.csv',index=False)

In [36]:
evals_y.to_csv('evals_y.csv',index=False)

In [ ]:
print("LGB test")
clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
    max_depth=-1, n_estimators=10000, objective='binary',
    subsample=0.65, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=-1
)
clf.fit(train_x, train_y, eval_set=[(evals_x, evals_y)], eval_metric='auc',early_stopping_rounds=100)
res['score'] = clf.predict_proba(test_x)[:,1]
res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
res.to_csv('./submission2.csv', index=False)
os.system('zip ./baseline2.zip ./submission2.csv')

In [49]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os

In [50]:
train_x=sparse.load_npz('train_x.npz')

In [51]:
evals_x=sparse.load_npz('evals_x.npz')

In [52]:
res=pd.read_csv('res.csv')

In [53]:
train_y=pd.read_csv('train_y.csv')

In [54]:
evals_y=pd.read_csv('evals_y.csv')

In [ ]:
test_x=sparse.load_npz('test_x.npz')

In [14]:
train_x.shape

(7039051, 419652)

In [15]:
train_y.shape

(7039050, 1)

In [13]:
train_x, evals_x = train_test_split(train_x,test_size=0.2, random_state=2018)

In [14]:
train_x

<7039051x419652 sparse matrix of type '<class 'numpy.float64'>'
	with 469401979 stored elements in Compressed Sparse Row format>

In [32]:
train=data[data.label!=-1]

train_y=train.pop('label')


In [34]:
train_y.shape


(7039051,)

In [33]:
train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)


In [35]:
train_y.to_csv('train_y.csv',index=False)

In [47]:
train_=pd.read_csv('train_y.csv')